## Attendance List Inspector

This notebook looks for two files:

1. an `apprentices.csv` that contains the columns `name`, `email` and `cohort`, and lists **all** the apprentices you may want to track attendance for. This should not need to change once set up.
   - *Note:* Include apprentices from **all** your cohorts here - just make sure their cohort is listed in the `cohort` column, and this will be filtered for the cohort you actually want to inspect attendance for below.
2. a file that contains the actual Teams attendance report you want to inspect. This can be obtained by downloading the report from Teams, and saving it into the same folder as this notebook.
   - *Note:* You can rename the file to anything you want or leave it with the default name Teams gives it - either way, enter the filename into the `FILE` parameter in the first code cell below, remembering the `.csv` extension.

See the included CSVs for examples. You can just copy and paste directly into `apprentice.csv` with your own apprentices (don't forget to remove the example rows!) and save over `attendance.csv` with the latest Teams report each time to make things easier.

Once both CSV files are in place, just run all the code below.

**Tip:** Enter the cohort name exactly as it appears in the `apprentices.csv` file in the variable `COHORT` in the first cell. For easier use, you can enter multiple `COHORT` code lines, one for each cohort you coach, and just comment / uncomment them to change the filter, rather than needing to type them out each time - example given below!

In [ ]:
# ==========
# PARAMETERS
# ==========

COHORT = 'First Cohort'
# COHORT = 'Second Cohort'

FILE = "attendance.csv"

In [ ]:
import pandas as pd
import re
import datetime

# read in all apprentice names, emails and cohorts
apps = pd.read_csv('apprentices.csv')
apps['email'] = apps['email'].str.lower()
apps.rename(columns = {'name': 'app_name'}, inplace = True)

# filter for a certain cohort in your apprentices list
apps = apps[apps['cohort'] == COHORT].copy()

# read in attendance report data and just keep name and email columns
attn = pd.read_csv(FILE, encoding = 'utf_16_le', sep = '\t', skiprows = 9, engine = 'python')
split_row = attn[attn['First Join'].isnull()].index.values[0]
attn = attn.iloc[0:split_row].copy()
attn.columns = attn.columns.str.lower()
attn.rename(columns = {'name': 'teams_handle'}, inplace = True)
attn['email'] = attn['email'].str.lower()
attn['email'].fillna('no email', inplace = True)
attn['attended'] = 'yes'
attn.drop_duplicates(inplace = True)

# merge
all_attn = apps.merge(attn, on = ['email'], how = 'outer')
all_attn['attended'].fillna('no', inplace = True)

## Did Not Attend

The following apprentices did **not** attend:

In [ ]:
all_attn[all_attn['attended'] == 'no'][['app_name']].sort_values(by = ['app_name'])

## Attended, No Email

The following apprentices attended but did not provide an email to Teams, and so need to be manually checked:

In [ ]:
all_attn[all_attn['email'] == 'no email'][['teams_handle']]

## Attended

The following apprentices attended:

In [ ]:
all_attn[all_attn['attended'] == 'yes'][['app_name', 'cohort']].dropna()

## All

These are the apprentices in the cohort, with their attendance status:

In [ ]:
all_attn[['app_name', 'attended']].dropna().style.applymap(lambda x: 'color: #b91c1c;font-weight:bold;' if x == 'no' else 'color: #16a34a;font-weight:bold;', subset = ['attended'])

## Stats

In [ ]:
stats = all_attn[['app_name', 'attended']].dropna().copy()

stats['attended'] = stats['attended'].map({'yes': 1, 'no': 0})

print(f"Total attendance: {stats['attended'].sum()} out of {len(stats)} ({round(stats['attended'].sum() / len(stats), 2) * 100}%)")

## Save to file

**Optional.**

Run the cell below to save the full attendance report to a CSV, with today's date added as a column. This could be used later to concat with other attendance reports for further analysis.

**Note:** If running this report on a different date to the workshop date, you can also change the `date` column below to another date by uncommenting and entering a custom date string in the format `yyyy-mm-dd` into the `WORKSHOP_DATE` variable below.

In [ ]:
WORKSHOP_DATE = datetime.datetime.now().strftime("%Y-%m-%d")
# WORKSHOP_DATE = "23-07-10"

to_save = all_attn[['app_name', 'attended']].dropna().copy()

to_save['date'] = pd.to_datetime(WORKSHOP_DATE)

# optional - add more metadata to this dataset
# this could also be joined later using a simple lookup table and merging on date

# to_save['module'] = 1
# to_save['workshop_title'] = "Data in Industry"
# to_save['day'] = 1

# save to CSV

filename = WORKSHOP_DATE + "_" + COHORT + ".csv"
to_save.to_csv(filename)


## TEST ZONE

**Note:** This is some extra code I've been trying out for my own use. Feel free to use, tweak, comment out or delete it as you like!

The code below will read multiple Teams reports saved into a sub-directory called `teams_reports` that exists in the same folder as this notebook. It will then show attendance info across all the dates given in those reports. Be aware:

1. For this to work, do not change the filename of the Teams report when you save it (as it contains the date info used)
2. This does not take into account missing email addresses on Teams as the above code does, so will need to be double checked

In [ ]:
# RAW TEAMS - DIRECTORY VERSION
from os import listdir
from os.path import isfile, join

dir = 'teams_reports'

# read in all files in subdir containing Teams reports
files = [f for f in listdir(dir) if isfile(join(dir, f))]

# blank frame for storage
attn_dates = pd.DataFrame()

# loop through each file in subdir and process date from filename,
# extract attendance information and concat with storage frame,
# and process attendance information
for file in files:
    match = re.search(r'\d+-\d{2}-\d{2}', file)
    date = datetime.datetime.strptime(match.group(), '%m-%d-%y')

    df = pd.read_csv(dir + '/' + file, encoding = 'utf_16_le', sep = '\t', skiprows = 9, engine = 'python')

    split_row = df[df['First Join'].isnull()].index.values[0]
    df = df.iloc[0:split_row].copy()

    df.columns = df.columns.str.lower()
    df.rename(columns = {'name': 'teams_handle'}, inplace = True)
    df['email'] = df['email'].str.lower()
    df['email'].fillna('no email', inplace = True)
    df['attended'] = 'yes'
    df.drop_duplicates(inplace = True)

    df = apps.merge(df, on = ['email'], how = 'outer')

    df['date'] = date
    df['attended'].fillna('no', inplace = True)

    attn_dates = pd.concat([attn_dates, df])

# pivot dates wide
attn_dates_wide = attn_dates[['app_name', 'attended', 'date']].dropna().pivot(index = 'app_name', columns = 'date', values = 'attended').fillna('no').reset_index().copy()

# create boolean list for subset of style applied, based on number of columns in df
cols = len(attn_dates_wide.columns) - 1

subset = [False]
while cols > 0:
    subset.append(True)
    cols -= 1

# style output for yes / no
attn_dates_wide.style.applymap(lambda x: 'color: #b91c1c;font-weight:bold;' if x == 'no' else 'color: #16a34a;font-weight:bold;', subset = subset)